# 1. Import dependencies

In [1]:
import copy
import cv2 as cv
import csv
import mediapipe as mp
import numpy as np
import itertools
import os

from collections import deque

# 2. Initialization

## a. Initialize variables and models

In [2]:
# Initialize Mediapipe's hand model 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(
    static_image_mode=True,   
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.2,
    model_complexity=0
)

# Initialize main directory 
path = os.getcwd()
main_directory = os.path.dirname(path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## b. Initialize functions

In [3]:
# Calculate landmark list 
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
# Pre-processing calculated landmarks 
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = base_y - temp_landmark_list[index][1]

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

## c. Initialize dataset template function

In [130]:
# Initialize number of hand keypoints and alphabet class
num_coords = 21
alphabet = "Z"

In [131]:
# Initialize .csv file
def init_csv(feature):
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val)]

    with open(main_directory +'/dataset/csv_pre_combined/keypoints_{}.csv'.format(feature), mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

## d. Initialize dataset logging function

In [132]:
def logging_csv(feature, landmark_list):

    # csv_path = main_directory + '/dataset/csv_pre_combined/keypoints_{}.csv'.format(feature)
    csv_path = main_directory + '/dataset/keypoints_combined.csv'
    
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([feature, *landmark_list])

    return

## e. Initialize image location

In [133]:
IMAGE_FILES = ['SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (1).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (2).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (3).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (4).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (5).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (6).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (7).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (8).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (9).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (10).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (11).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (12).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (13).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (14).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (15).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (16).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (17).jpg'.format(alphabet, alphabet), 
               'SIBI_datasets_LEMLITBANG_SIBI_R_90.10_RAW/{}/{} (18).jpg'.format(alphabet, alphabet)]

# 3. Capture, re-normalize & write landmark keypoints into dataset

In [134]:
for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv.flip(cv.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    
    if not results.multi_hand_landmarks:
        continue
    
    image_height, image_width, _ = image.shape
    debug_image = copy.deepcopy(image)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Convert pre-normalized landmark keys into pixels numbering
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)
            
            # Convert into relative coordinates / normalize keys from wrist point
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            
            # Write/log received keypoints into dataset
            logging_csv(alphabet, pre_processed_landmark_list)
            print(alphabet, logging_csv)
            # Visualize complete hand landmarks
            mp_drawing.draw_landmarks(
                debug_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
Z <function logging_csv at 0x7fd3765db880>
